In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path

from helpers import model1, model2, model3, filter_data, onehotencode, extract_features_data, simple_mlp, model4, conv

C:\Users\mleon\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [4]:
training_data_path = Path('..')/ 'data' / 'training_data' 
get_sensor_data = lambda x: training_data_path / 'respondent{}_sensor_data.csv'.format(x)
get_label = lambda x: training_data_path /'respondent{}_label.csv'.format(x)

get_sensor_data(1), get_label(1)

(WindowsPath('../data/training_data/respondent1_sensor_data.csv'),
 WindowsPath('../data/training_data/respondent1_label.csv'))

## Read data

In [5]:
num_respondents = 32
data_dict = {}
label_dict = {}
for num in range(1, num_respondents+1):
    data_dict['respondent_{}'.format(num)]  = pd.read_csv(get_sensor_data(num), index_col=[0,1])
    label_dict['respondent_{}'.format(num)] = pd.read_csv(get_label(num), index_col=[0])

In [6]:
sensor_data = pd.concat(data_dict)
display(sensor_data.head())

labels = pd.concat(label_dict)
display(labels.head(10))

POz (Raw)   Fz (Raw)   Cz (Raw)  C3 (Raw)  \
respondent_1 trailer_1 0  -2.176072  -3.730452  -6.648115  7.906390   
                       1 -17.308790 -19.317900 -24.012070 -3.379530   
                       2 -17.308790 -19.317900 -24.012070 -3.379530   
                       3 -17.308790 -19.317900 -24.012070 -3.379530   
                       4 -16.074980 -19.023580 -28.106670  2.857648   

                           C4 (Raw)  F3 (Raw)   F4 (Raw)  P3 (Raw)   P4 (Raw)  
respondent_1 trailer_1 0   7.838735  27.81043  -1.944833  4.576408  -0.719508  
                       1  -9.873293  12.86720 -18.772840  0.447505 -16.476750  
                       2  -9.873293  12.86720 -18.772840  0.447505 -16.476750  
                       3  -9.873293  12.86720 -18.772840  0.447505 -16.476750  
                       4 -27.548620  17.50465 -29.356010 -0.545797  -4.480889

labels
respondent_1 trailer_1     9.0
             trailer_2     5.0
             trailer_3     7.0
             trailer_4     1.0
             trailer_5     5.0
             trailer_6     6.0
respondent_2 trailer_1     6.0
             trailer_2     8.0
             trailer_3     6.0
             trailer_4     6.0

# Movie Rating Classification (2 Classes)

In [10]:
tmp_labels = labels.copy()


conditions = [tmp_labels["labels"].isin([1,2,3,4,5]),
             tmp_labels["labels"].isin([6,7,8,9,10])]

choices = ["Dislike", "Like"]

tmp_labels['Label'] = np.select(conditions, choices)

tmp_labels.drop("labels", inplace=True, axis=1)

In [11]:
tmp_labels

Label
respondent_1  trailer_1     Like
              trailer_2  Dislike
              trailer_3     Like
              trailer_4  Dislike
              trailer_5  Dislike
...                          ...
respondent_32 trailer_2  Dislike
              trailer_3     Like
              trailer_4  Dislike
              trailer_5     Like
              trailer_6     Like

[192 rows x 1 columns]

In [12]:
tmp_labels = pd.get_dummies(tmp_labels)

tmp_labels

Label_Dislike  Label_Like
respondent_1  trailer_1              0           1
              trailer_2              1           0
              trailer_3              0           1
              trailer_4              1           0
              trailer_5              1           0
...                                ...         ...
respondent_32 trailer_2              1           0
              trailer_3              0           1
              trailer_4              1           0
              trailer_5              0           1
              trailer_6              0           1

[192 rows x 2 columns]

In [13]:
#sensor_data.groupby(level=[0,1]).apply(lambda x: np.isnan(x.values).sum())
print('shape with nans: ', sensor_data.shape)
sensor_data.dropna(inplace=True)
print('shape without nans: ', sensor_data.shape)

shape with nans:  (17376893, 9)
shape without nans:  (17376862, 9)


In [14]:
num_respondents = num_respondents
num_trailer = 6
num_time = 15360 # 65000
num_channels = 9
num_labels = 2

all_data = np.empty(shape=(num_respondents,num_trailer,num_time,num_channels))
all_labels_rating = np.empty(shape=(num_respondents,num_trailer, num_labels))

for num_r in range(1,num_respondents+1):
    for num_t in range(1,num_trailer+1):
        df_ = sensor_data.xs([f'respondent_{num_r}', f'trailer_{num_t}'], level=[0,1]) 
        label = tmp_labels.xs([f'respondent_{num_r}', f'trailer_{num_t}'], level=[0,1]) 
        all_data[num_r-1, num_t-1] = df_.values[:num_time]
        all_labels_rating[num_r-1, num_t-1] = label.values

print(all_data.shape, all_labels_rating.shape)

(32, 6, 15360, 9) (32, 6, 2)


In [15]:
all_labels_rating = np.vstack(all_labels_rating)

# Conv1D

## Training LSTM

In [49]:
model = conv()
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_45 (Conv1D)           (None, 15360, 20)         200       
_________________________________________________________________
dropout_75 (Dropout)         (None, 15360, 20)         0         
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, 7680, 20)          0         
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 7678, 40)          2440      
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 3839, 40)          0         
_________________________________________________________________
dropout_76 (Dropout)         (None, 3839, 40)          0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 153560)          

In [16]:
np.argmax(all_labels_rating, axis=1)

from sklearn.utils import compute_class_weight

classWeight = compute_class_weight('balanced', np.unique(np.argmax(all_labels_rating, axis=1)
), np.argmax(all_labels_rating, axis=1)
) 
classWeight_ratings = dict(enumerate(classWeight))

classWeight_ratings

{0: 1.92, 1: 0.676056338028169}

In [17]:
from helpers import extract_features_data
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import keras
X = all_data.copy()
#y = all_labels.copy()

X = np.vstack(X)

X = filter_data(X)

#y = np.vstack(to_categorical(y, num_classes=6, dtype="int32"))


In [18]:
from mne.decoding import Scaler

sc = Scaler(scalings="mean")

X_scaled = sc.fit_transform(np.transpose(X, axes = [0,2,1]))

X_scaled = np.transpose(X_scaled, axes = [0,2,1])

X_scaled.shape

(192, 15360, 9)

In [18]:
X_train, X_test,y_train, y_test = train_test_split(X,all_labels_rating, test_size = 0.1, shuffle = True)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [19]:
callback = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20)



In [44]:
model = KerasClassifier(conv ,epochs=100, batch_size=60, verbose=0)

# class_weight = classWeight_ratings,validation_data = (X_val, y_val)
# class_weight=classWeight, callbacks=[callback], validation_data = (X_val, y_val)

In [25]:
y_flatten = np.argmax(all_labels_rating, axis= 1)

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

scoring = {'acc': 'accuracy',
           "F1": "f1"
          }

scores = cross_validate(model, X_scaled, y_flatten, cv=5, scoring = scoring, fit_params = {"class_weight": classWeight_ratings})

In [47]:
scores

{'fit_time': array([32.7152369 , 33.0822928 , 33.09768963, 33.92539644, 33.82549763]),
 'score_time': array([0.20245767, 0.17722487, 0.20074439, 0.17949104, 0.17356515]),
 'test_acc': array([0.69230769, 0.61538462, 0.71052632, 0.81578947, 0.89473684]),
 'test_F1': array([0.80645161, 0.74576271, 0.81967213, 0.89552239, 0.93939394])}

In [46]:
np.mean(scores["test_acc"])

0.745748987854251

In [48]:
np.mean(scores["test_F1"])

0.8413605566737627

In [18]:
scores

{'fit_time': array([76.56728458, 72.78822541, 73.71403408, 73.15319777, 73.40411758]),
 'score_time': array([0.34551239, 0.21043754, 0.48876929, 0.20548153, 0.21246028]),
 'test_acc': array([0.66666667, 0.56410256, 0.71052632, 0.76315789, 0.73684211]),
 'test_F1': array([0.77966102, 0.72131148, 0.81355932, 0.86153846, 0.82758621])}

# With Band Energy Feature Extraction and Ensemble

In [20]:
X_features = extract_features_data(X, "energy_freq_bands", funcs_params = {"energy_freq_bands__freq_bands": np.array([.5, 4, 8, 13, 35])})



In [26]:
X_features

energy_freq_bands                                            \
            ch0_band0     ch0_band1     ch0_band2     ch0_band3   
0        20854.900171  18699.948940  37863.288041  1.570576e+06   
1        41688.333526  26635.590427  77009.496668  9.226720e+05   
2        13822.238137  24934.447349  56345.467423  1.048633e+06   
3        20226.528125  21705.481564  59224.709309  9.880138e+05   
4        13798.192998  26215.926823  64301.032986  1.375484e+06   
..                ...           ...           ...           ...   
187      22583.065257  23196.919956  41982.048586  1.664045e+06   
188      48018.992709  24125.477391  44535.577689  2.172591e+06   
189      23286.137194  22164.028872  44698.969896  2.071526e+06   
190      40577.459824  26485.385049  55620.656241  1.995208e+06   
191      16009.577517  26894.540752  50913.806488  2.057228e+06   

                                                                            \
         ch1_band0     ch1_band1     ch1_band2     ch1_band3     ch2_band0   
0     47082.781049  24564.721627  36893.157717  1.822657e+06  31669.123084   
1    105095.950034  65090.939773  44236.027859  1.065557e+06  47657.955791   
2     53570.153782  36878.657184  53917.909069  1.191501e+06  16823.336757   
3     57339.968383  41737.397779  52736.544662  1.153407e+06  25677.923905   
4     45553.136362  37824.195706  47352.959865  1.562937e+06  16866.222268   
..             ...           ...           ...           ...           ...   
187   36112.910073  35213.515951  55706.180304  2.230909e+06  30930.688625   
188   65486.470137  35102.489892  52099.727022  2.859942e+06  69234.394626   
189   43173.167871  32018.371061  55187.950022  2.716797e+06  34650.698109   
190   55348.136333  33012.266557  58025.034279  2.617743e+06  52536.125436   
191   34046.627176  35616.006058  50212.575561  2.636138e+06  28828.863477   

                   ...                                            \
        ch2_band1  ...     ch6_band2     ch6_band3     ch7_band0   
0    19151.409840  ...  41183.501242  1.923355e+06   7251.200606   
1    23257.710667  ...  53337.437743  1.170431e+06  35023.668352   
2    22581.194620  ...  57941.830619  1.387862e+06   9778.984177   
3    25954.706091  ...  58868.696025  1.304686e+06  22099.050527   
4    24902.926143  ...  52828.742912  1.638172e+06   8964.150571   
..            ...  ...           ...           ...           ...   
187  29255.447892  ...  53790.497397  2.162264e+06  10388.815944   
188  33038.087224  ...  54787.028077  2.787827e+06  32423.894896   
189  28178.631253  ...  52149.960110  2.643691e+06  14308.413738   
190  33097.758210  ...  57990.654838  2.515602e+06  22429.899551   
191  32264.228870  ...  47381.298604  2.527717e+06  13844.295449   

                                                                           \
        ch7_band1     ch7_band2     ch7_band3     ch8_band0     ch8_band1   
0    12779.302208  31602.987167  1.432127e+06  21561.554639  13065.654070   
1    18906.641340  52297.107331  8.164647e+05  29840.751964  14018.061803   
2    21158.563559  48638.560653  9.420274e+05   8322.654511  15503.096686   
3    23002.327201  52776.219884  9.070597e+05  14394.670427  16030.521735   
4    19843.700747  61832.051183  1.542027e+06   8422.777522  19058.385587   
..            ...           ...           ...           ...           ...   
187  27113.631720  47802.063772  1.061409e+06  20794.797182  17676.880072   
188  29798.334048  47350.898197  1.411443e+06  43209.044394  22126.283659   
189  25246.237491  44844.038785  1.362143e+06  16817.462279  19013.748875   
190  26578.708701  58174.323888  1.297626e+06  33808.429946  24339.692930   
191  31947.170285  56051.393175  1.330501e+06  15156.387418  21292.790280   

                                 
        ch8_band2     ch8_band3  
0    29592.629645  1.321403e+06  
1    38633.204411  6.544255e+05  
2    39702.357993  8.010054e+05  
3    37263.294168  7.120677e+05  
4    52780.146754  1.31302

In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

clf1 = RandomForestClassifier(class_weight=classWeight_ratings)
clf2 = LogisticRegression(class_weight=classWeight_ratings, max_iter=10000)
clf3 = SVC(probability=True, class_weight= classWeight_ratings)




voting_clf = VotingClassifier(estimators = [("rf",clf1), ("LogReg",clf2), ("SVC",clf3)], voting="soft")


scores_ensemble = cross_validate(voting_clf, X_features, y_flatten, cv=5, scoring = scoring)

In [25]:
np.mean(scores_ensemble["test_acc"])

0.6354925775978407

# RNN with Leave-one-out Cross-Validation

In [ ]:
if False:    
    from sklearn.preprocessing import StandardScaler
    from keras.wrappers.scikit_learn import KerasClassifier
    from keras.utils import to_categorical 

    model = KerasClassifier(model1, epochs=20, batch_size=30, verbose=1)
    for i in range(all_data.shape[0]):

        X_train = all_data.copy()
        y_train = all_labels.copy()



        X_test = X_train[i].copy()
        y_test = y_train[i].copy()

        X_train = np.delete(X_train, i, axis=0)
        y_train = np.delete(y_train, i, axis=0)



        X_train = np.vstack(X_train)

        X_train = filter_data(X_train)
        X_test = filter_data(X_test)

        scalers = {}
        for i in range(X_train.shape[2]):
            scalers[i] = StandardScaler()
            X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])

        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])     

        y_test = to_categorical(y_test, num_classes = 6, dtype="int32")
        y_train = np.vstack(to_categorical(y_train, num_classes=6, dtype="int32"))


        model.fit(X_train, y_train, validation_data=(X_test, y_test), shuffle=True, use_multiprocessing=True)

        print(f'model score: {model.score(X_test, y_test)}')